**ContrastCoding tech that Compares mean of dependent var in each cat to ref level thereby bring out constrast btw diff levels in cat**

In [1]:
#5th, useful for anova test(assumption that features are independent) constrast coding technique

import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [2]:
iris_data = pd.read_csv("./datasets/iris.csv")

iris_data.sample(10)

,Species,sepal_length,sepal_width,petal_length,petal_width
89,Iris-versicolor,5.5,2.5,4.0,1.3
48,Iris-setosa,5.3,3.7,1.5,0.2
124,Iris-virginica,6.7,3.3,5.7,2.1
129,Iris-virginica,7.2,3.0,5.8,1.6
102,Iris-virginica,7.1,3.0,5.9,2.1
18,Iris-setosa,5.7,3.8,1.7,0.3
87,Iris-versicolor,6.3,2.3,4.4,1.3
31,Iris-setosa,5.4,3.4,1.5,0.4
149,Iris-virginica,5.9,3.0,5.1,1.8
125,Iris-virginica,7.2,3.2,6.0,1.8


In [3]:
# drop cols that we don't need, aim is to use cat to predict petal length
iris_data.drop(columns = ["sepal_length", "sepal_width", "petal_width"], inplace = True)

iris_data.sample(6)

,Species,petal_length
93,Iris-versicolor,3.3
58,Iris-versicolor,4.6
120,Iris-virginica,5.7
47,Iris-setosa,1.4
3,Iris-setosa,1.5
138,Iris-virginica,4.8


In [4]:
iris_data.describe()

,petal_length
count,150.000000
mean,3.758667
std,1.764420
min,1.000000
25%,1.600000
50%,4.350000
75%,5.100000
max,6.900000


In [5]:
# gets mean for each mat
iris_species_mean = iris_data.groupby(by="Species").mean()

iris_species_mean

,petal_length
Species,
Iris-setosa,1.464
Iris-versicolor,4.260
Iris-virginica,5.552


In [6]:
# using iris_setosa as ref value and calc coeffor versicolor manually, setosa is used as ref level for this task
iris_species_mean.loc["Iris-versicolor"]["petal_length"] - \
        iris_species_mean.loc["Iris-setosa"]["petal_length"]  

2.796

In [7]:
iris_species_mean.loc["Iris-virginica"]["petal_length"] - \
        iris_species_mean.loc["Iris-setosa"]["petal_length"] 

4.087999999999999

In [8]:
# treatment coding(dummy coding) usings stats
mod = ols("petal_length ~ Species",  #uses iris_setosa as ref level by default
         data = iris_data)

res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           petal_length   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     1179.
Date:                Tue, 12 Jan 2021   Prob (F-statistic):           3.05e-91
Time:                        13:35:14   Log-Likelihood:                -84.840
No. Observations:                 150   AIC:                             175.7
Df Residuals:                     147   BIC:                             184.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      1.4640      0.061     24.057      0.000       1.344       1.584
Species[T.Iris-versicolor]     2.7960      0.086     32.488      0.000       2.626       2.966
Species[T.Iris-virginica]      4.0880      0.086     47.500      0.000       3.918       4.258
==============================================================================
Omnibus:                        4.393   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.111   Jarque-Bera (JB):                5.370
Skew:                           0.121   Prob(JB):                       0.0682
Kurtosis:                       3.895   Cond. No.                         3.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Simple Effect Coding**

In [10]:
from patsy.contrasts import ContrastMatrix

def _name_levels(prefix, levels):
    return ["[%s%s]" % (prefix, level) for level in levels]

In [11]:
# class that does simpleEffect coding since not available by default
# class is just copy and use not fully understood.

class Simple(object):
    def _simple_contrast(self, levels):
        nlevels = len(levels)
        contr = -1./nlevels * np.ones((nlevels, nlevels-1))
        contr[1:][np.diag_indices(nlevels-1)] = (nlevels-1.)/nlevels
        return contr

    def code_with_intercept(self, levels):
        contrast = np.column_stack((np.ones(len(levels)),
                                    self._simple_contrast(levels)))
        return ContrastMatrix(contrast, _name_levels("Simp.", levels))

    def code_without_intercept(self, levels):
        contrast = self._simple_contrast(levels)
        return ContrastMatrix(contrast, _name_levels("Simp.", levels[:-1]))

In [12]:
mod = ols("petal_length ~ C(Species, Simple)", # uses simple effect coding class to code data vua "Simple"
          data = iris_data)

res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           petal_length   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     1179.
Date:                Tue, 12 Jan 2021   Prob (F-statistic):           3.05e-91
Time:                        13:43:21   Log-Likelihood:                -84.840
No. Observations:                 150   AIC:                             175.7
Df Residuals:                     147   BIC:                             184.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    3.7587      0.035    106.978      0.000       3.689       3.828
C(Species, Simple)[Simp.Iris-setosa]         2.7960      0.086     32.488      0.000       2.626       2.966
C(Species, Simple)[Simp.Iris-versicolor]     4.0880      0.086     47.500      0.000       3.918       4.258
==============================================================================
Omnibus:                        4.393   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.111   Jarque-Bera (JB):                5.370
Skew:                           0.121   Prob(JB):                       0.0682
Kurtosis:                       3.895   Cond. No.                         3.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Simple Effect Coding Manually**

In [13]:
levels = [0, 1, 2] # since 3 cat

In [15]:
contrast_with_intercept = Simple().code_with_intercept(levels)
contrast_with_intercept.matrix  # 1 is for the intercept

array([[ 1.        , -0.33333333, -0.33333333],
       [ 1.        ,  0.66666667, -0.33333333],
       [ 1.        , -0.33333333,  0.66666667]])

In [16]:
contrast_without_intercept = Simple().code_without_intercept(levels)
contrast_without_intercept

ContrastMatrix(array([[-0.33333333, -0.33333333],
                      [ 0.66666667, -0.33333333],
                      [-0.33333333,  0.66666667]]),
               ['[Simp.0]', '[Simp.1]'])

In [18]:
# labels encode species cat data
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

In [22]:
iris_data_enc = iris_data.copy()

iris_data_enc["species_encoded"] = label_encoder.fit_transform(iris_data.Species)

In [23]:
label_encoder.classes_

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [28]:
# does simple effect coding with cat at index 0 i.e setoas as ref value
iris_data_contrast_setosa = contrast_without_intercept.matrix[iris_data_enc.species_encoded - 0, :]
iris_data_contrast_setosa[:5]

array([[-0.33333333, -0.33333333],
       [-0.33333333, -0.33333333],
       [-0.33333333, -0.33333333],
       [-0.33333333, -0.33333333],
       [-0.33333333, -0.33333333]])

In [26]:
# same as above but with cols for intercept
iris_data_contrast_setosa = contrast_with_intercept.matrix[iris_data_enc.species_encoded - 0, :]
iris_data_contrast_setosa[:5]

array([[ 1.        , -0.33333333, -0.33333333],
       [ 1.        , -0.33333333, -0.33333333],
       [ 1.        , -0.33333333, -0.33333333],
       [ 1.        , -0.33333333, -0.33333333],
       [ 1.        , -0.33333333, -0.33333333]])

In [29]:
# gets df to be used with reg model
iris_data_contrast = pd.DataFrame(iris_data_contrast_setosa,
                                 columns = ["Iris-Versicolor", "Iris-Versicolor"])


iris_data_contrast.sample(6)

,Iris-Versicolor,Iris-Versicolor
89,0.666667,-0.333333
139,-0.333333,0.666667
86,0.666667,-0.333333
83,0.666667,-0.333333
62,0.666667,-0.333333
147,-0.333333,0.666667


In [30]:
# concat contrast encoded with main df
iris_data_enc = pd.concat([iris_data,  iris_data_contrast], axis = "columns")

iris_data_enc.sample(8)

,Species,petal_length,Iris-Versicolor,Iris-Versicolor
15,Iris-setosa,1.5,-0.333333,-0.333333
21,Iris-setosa,1.5,-0.333333,-0.333333
0,Iris-setosa,1.4,-0.333333,-0.333333
16,Iris-setosa,1.3,-0.333333,-0.333333
59,Iris-versicolor,3.9,0.666667,-0.333333
54,Iris-versicolor,4.6,0.666667,-0.333333
47,Iris-setosa,1.4,-0.333333,-0.333333
38,Iris-setosa,1.3,-0.333333,-0.333333


In [31]:
X = iris_data_enc.drop(columns = ["Species", "petal_length"])
y = iris_data_enc.petal_length

In [32]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()

linear_model.fit(X, y)

print("Train score: ", linear_model.score(X, y))  # same as score in stats model

Train score:  0.9413189735606261


In [34]:
linear_model.coef_

array([2.796, 4.088])